<a href="https://colab.research.google.com/github/kashish45/Machine-Learning-Projects/blob/master/Machine_Hack_Computer_Vision_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://machinehack-be.s3.amazonaws.com/computer_vision_classic_weekend_hackathon_11/Data.zip

--2021-02-11 06:30:51--  https://machinehack-be.s3.amazonaws.com/computer_vision_classic_weekend_hackathon_11/Data.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.62.44
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.62.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 315463646 (301M) [application/octet-stream]
Saving to: ‘Data.zip’

Data.zip            100%[===================>] 300.85M  12.8MB/s    in 25s     

2021-02-11 06:31:18 (12.0 MB/s) - ‘Data.zip’ saved [315463646/315463646]



In [2]:
#importing libraries
import pandas as pd
from tensorflow.keras.optimizers import RMSprop,Adam,Adagrad
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras import Model
from os import getcwd
from keras.layers import Dense, Activation, BatchNormalization
import tensorflow as tf

In [3]:
!unzip -q Data.zip

In [17]:
TRAIN="/content/Data/Train"
BATCH_SIZE=16

In [18]:

train_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range = 40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    validation_split=0.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN,
    subset = 'training',
    batch_size = BATCH_SIZE,
    class_mode='categorical',
    target_size = (224,224)
)


validation_generator = train_datagen.flow_from_directory(
    TRAIN,
    subset = 'validation',
    batch_size = BATCH_SIZE,
    class_mode='categorical',
    target_size=(224,224)
)   

Found 7578 images belonging to 2 classes.
Found 1893 images belonging to 2 classes.


In [19]:
from keras.applications import DenseNet201


In [20]:
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor = 'val_accuracy', patience = 15, verbose=0, mode='max')
mcp_save_xc = ModelCheckpoint(filepath = 'mdl_wts.hdf5', save_best_only = True, monitor = 'val_accuracy', mode = 'max')
mcp_save_ens = ModelCheckpoint(filepath = 'mdl_wts_en.h5', save_best_only = True, monitor = 'val_accuracy', mode = 'max')
mcp_save_ens1 = ModelCheckpoint(filepath =' mdl_wts_en1.h5', save_best_only = True, monitor = 'val_loss', mode = 'min')
reduce_lr_loss = ReduceLROnPlateau(monitor = 'val_loss', patience=2, verbose=1, min_delta = 1e-4, mode = 'min')

In [21]:
pre_model = DenseNet201( input_shape = (224,224,3), include_top = False , weights = "imagenet")

In [22]:
for layer in pre_model.layers:
  layer.trainable = True

In [23]:
last_layer = pre_model.get_layer('relu')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 1920)


In [24]:
#for xception
x = layers.Flatten()(last_output)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1024, activation = "relu")(x)

x = layers.Dropout(0.4)(x)

x = layers.BatchNormalization()(x)

x = layers.Dense(2, activation = "softmax")(x)

model_xception = Model(pre_model.input,x)

model_xception.compile(optimizer = Adagrad(lr=0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy']
             )

In [25]:
history_xc = model_xception.fit(
      train_generator,  
      callbacks=[ mcp_save_xc, reduce_lr_loss,earlyStopping],
      validation_data = validation_generator,
      epochs = 20,
      verbose = 1,
 )

Epoch 1/20
474/474 [==============================] - 247s 485ms/step - loss: 0.2337 - accuracy: 0.9102 - val_loss: 0.0442 - val_accuracy: 0.9826
Epoch 2/20
474/474 [==============================] - 222s 469ms/step - loss: 0.0711 - accuracy: 0.9732 - val_loss: 0.0337 - val_accuracy: 0.9857
Epoch 3/20
474/474 [==============================] - 222s 468ms/step - loss: 0.0679 - accuracy: 0.9767 - val_loss: 0.0339 - val_accuracy: 0.9878
Epoch 4/20
474/474 [==============================] - 223s 470ms/step - loss: 0.0599 - accuracy: 0.9789 - val_loss: 0.0276 - val_accuracy: 0.9878
Epoch 5/20
474/474 [==============================] - 223s 470ms/step - loss: 0.0571 - accuracy: 0.9821 - val_loss: 0.0326 - val_accuracy: 0.9868
Epoch 6/20
474/474 [==============================] - 223s 471ms/step - loss: 0.0438 - accuracy: 0.9851 - val_loss: 0.0291 - val_accuracy: 0.9884

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 7/20
474/474 [======================

In [28]:
test_dir ='/content/Data/testing/'

In [50]:
test_datagen = ImageDataGenerator(rescale=1./255) 
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224,224),
        #classes=['test'],
        shuffle=False,
        class_mode = 'categorical')

Found 4059 images belonging to 1 classes.


In [51]:
predict = model_xception.predict(test_generator , steps=None )

In [52]:
prediction_cls_idx = predict.argmax(axis=-1)
prediction_cls_idx

array([1, 1, 0, ..., 1, 0, 1])

In [53]:
from zipfile import ZipFile as zip
import numpy as np
idx_to_cls = {v: k for k, v in train_generator.class_indices.items()}
prediction_cls= np.vectorize(idx_to_cls.get)(prediction_cls_idx)


In [54]:
filenames_to_cls = list((test_generator.filenames, prediction_cls))

In [55]:
import pandas as pd
data = pd.DataFrame(filenames_to_cls)
data = data.T
data.columns = ['File_Name', 'Category'] 

In [56]:
data['File_Name'] = data['File_Name'].str.replace('Test/','')

In [57]:
data.to_csv('output.csv', index = False)

In [58]:
data['Category']=data['Category'].replace('Dogs',1)
data['Category']=data['Category'].replace('Cats',0)

In [61]:
data.head(20)

,File_Name,Category
0,10005.jpg,1
1,10007.jpg,1
2,10008.jpg,0
3,10009.jpg,0
4,10011.jpg,0
5,10012.jpg,0
6,10013.jpg,0
7,10014.jpg,1
8,10018.jpg,0
9,10020.jpg,0


In [60]:
data.to_csv('o.csv', index = False)

In [62]:
data['Category'].value_counts()

1    2045
0    2014
Name: Category, dtype: int64